<a href="https://colab.research.google.com/github/LimonHalder/Numerical-Digit-Classifcations-Using-CNN/blob/master/firstKaggleSubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                          title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                     Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23           8951        277  1.0              
devansodariya/student-performance-data                       Student Performance Dataset                           7KB  2022-05-26 13:55:09           4613        152  0.9705882        
iamsouravbanerjee/software-professional-salaries-2022        Salary Dataset - 2022                               526KB  2022-06-15 17:13:05           1732         46  1.0              
paradisejoy/top-hits-spotify-from-20002019                   Top Hits Spoti

In [ ]:
!kaggle competitions download -c digit-recognizer

  0% 0.00/15.3M [00:00<?, ?B/s]
100% 15.3M/15.3M [00:00<00:00, 212MB/s]


In [ ]:
!unzip /content/digit-recognizer.zip

Archive:  /content/digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as ts
import tensorflow.keras as ks
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

pd.set_option('display.max_columns', 784)

In [ ]:

im_shape = (28, 28, 1)

out_df = pd.read_csv('/content/sample_submission.csv')
out_df.set_index('ImageId', inplace = True)

def prepare_data(df: pd.DataFrame, train = False):
    x_train = y_train = None
    
    if train:
        y_train = df['label']
        y_train = to_categorical(y_train, num_classes = 10)
        x_train = df.drop('label', axis=1, inplace=False)
    else:
        x_train = df
    
    # There is no semantic loss but convolutions **cores** will contain sane numbers
    x_train = x_train / 255
    
    # (rows, 28, 28, 1) 
    x_train = x_train.values.reshape(-1, *im_shape)
    
    return x_train, y_train

x_test, _        = prepare_data(pd.read_csv('/content/test.csv'))
x_train, y_train = prepare_data(pd.read_csv('/content/train.csv'),  train=True)

x_train_splitted, x_test_splitted, y_train_splitted, y_test_splitted = train_test_split(
    x_train, y_train, test_size = 0.2
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# augmentation setup
mnist_datagen = ImageDataGenerator(
        samplewise_center              = False, 
        featurewise_std_normalization  = False,  
        samplewise_std_normalization   = False,  
        zca_whitening                  = True,  
        horizontal_flip                = False,  
        vertical_flip                   = False,
    
        rotation_range                 = 10,  
        width_shift_range              = 0.1,  
        height_shift_range             = 0.1, 
)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
# learning options
epoch_count = 150
prefered_batch_size  = 256

model_options = {
    'optimizer': 'adam',
    'metrics':   'accuracy',                  
    'loss':      'categorical_crossentropy'  # awesome loss-function for multiclass classification
}
lrelu = lambda x: ks.layers.LeakyReLU(alpha = 0.01)(x)

# C part (2 convolutions|MaxPools(2x2))
conv_setup = {
    'kernel_size': 3,
    'strides':     1,               # we are getting them using kernel(size 5) with step 1
    'activation': 'relu'
}


lr_model = ks.Sequential()
# -----------------------------------------------------------------  # C1
lr_model.add(ks.layers.Conv2D(
    filters     = 64,               # 32 feature maps*
    input_shape = im_shape,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.Conv2D(
    filters     = 64,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.MaxPool2D()) # pool size is 2x2 by default
lr_model.add(ks.layers.Dropout(0.4))
lr_model.add(ks.layers.Conv2D(
    filters     = 128,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.Conv2D(
    filters     = 128,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.MaxPool2D())
lr_model.add(ks.layers.Dropout(0.4))

# -----------------------------------------------------------------  # Classifier part
lr_model.add(ks.layers.Flatten())   # 2d -> 1d(256 units)
lr_model.add(ks.layers.Dense(
    1024, activation = 'swish'
))


lr_model.add(ks.layers.Dropout(0.4))
lr_model.add(ks.layers.BatchNormalization())

# -----------------------------------------------------------------  # Output layer
lr_model.add(ks.layers.Dense(
    10, activation='softmax'        # 10 classes
))

# Assembly model
lr_model.compile(**model_options)
lr_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                        

In [ ]:
# train our model
lr_model.fit_generator(
    mnist_datagen.flow(x_train_splitted, y_train_splitted, batch_size = prefered_batch_size),
    verbose = 1,
    epochs = 100,
    steps_per_epoch= x_train_splitted.shape[0] // prefered_batch_size,
    validation_data = (x_test_splitted, y_test_splitted),    
)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


131/131 [==============================] - 23s 76ms/step - loss: 0.3680 - accuracy: 0.8844 - val_loss: 11.2494 - val_accuracy: 0.1099
Epoch 2/100
131/131 [==============================] - 10s 80ms/step - loss: 0.1087 - accuracy: 0.9658 - val_loss: 10.5254 - val_accuracy: 0.1099
Epoch 3/100
131/131 [==============================] - 13s 96ms/step - loss: 0.0834 - accuracy: 0.9744 - val_loss: 6.2014 - val_accuracy: 0.1531
Epoch 4/100
131/131 [==============================] - 10s 73ms/step - loss: 0.0666 - accuracy: 0.9801 - val_loss: 0.2558 - val_accuracy: 0.9237
Epoch 5/100
131/131 [==============================] - 10s 73ms/step - loss: 0.0568 - accuracy: 0.9808 - val_loss: 0.0357 - val_accuracy: 0.9879
Epoch 6/100
131/131 [==============================] - 9s 69ms/step - loss: 0.0513 - accuracy: 0.9841 - val_loss: 0.0270 - val_accuracy: 0.9912
Epoch 7/100
131/131 [==============================] - 9s 69ms/step - loss: 0.0496 - accuracy: 0.9849 - val_loss: 0.0243 - val_accuracy: 0.99

In [ ]:
# predict classes
y_preds = lr_model.predict(x_test)
y_preds_classified = np.argmax(y_preds, axis=1)
y_preds_classified[:10]

array([2, 0, 9, 0, 3, 7, 0, 3, 0, 3])

In [ ]:
# upload results 
out_df['Label'] = y_preds_classified
out_df.to_csv('outer.csv')

In [ ]:
p=pd.read_csv('outer.csv')
p

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
